In [37]:
from pyspark.sql import SparkSession
from pyspark.sql import Row
import pyspark.sql.functions as F
import pandas as pd
import datetime


spark = SparkSession \
            .builder \
            .config("spark.master", "local") \
            .config('spark.executor.cores', '8')\
            .config('spark.executor.memory', '2g')\
            .config('spark.driver.memory', '2g')\
            .config('spark.dynamicAllocation.minExecutors', '4')\
            .appName("spark_halltape") \
    .getOrCreate()


print("Активные Spark сессии:", 'http://localhost:4040/jobs')

Активные Spark сессии: http://localhost:4040/jobs


In [1]:
spark.stop()

NameError: name 'spark' is not defined

In [143]:
PATH = '/home/jovyan/source'

def debit_card(date):
    card = spark.read.csv(f"{PATH}/card.csv", header=True, sep=";")\
                .where(f''' load_date = "{date}" ''')
    
    status_card = spark.read.csv(f"{PATH}/Status_card.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    transactions = spark.read.csv(f"{PATH}/transactions.csv", header=True, sep=";")\
                        .where(f''' load_date = "{date}"  ''')
    
    
    first_trx = transactions.groupBy('card_num')\
                            .agg(F.min('transaction_datetime').alias('transaction_datetime'))
    
    
    first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
    
    dt_trx = first_trx_info.select('card_num', 'amount', '*')
    df_st = status_card.select('card_num', 'card_num_md5', 'status')
    
    result_df = dt_trx\
                    .join(df_st, "card_num", "inner")\
                    .join(card, "card_num_md5", "right")\
                    .drop('card_num_md5')
    
    final_df = result_df\
                    .where(''' card_num IS NOT NULL ''')\
                    .groupBy('card_num',
                             'transaction_datetime',
                             'status',
                             'card_order_dt',
                             'url',
                             'cookie')\
                    .agg(F.max('amount').alias('amt'))
    
    
    datamart = final_df\
                    .select('card_order_dt',
                            'card_num',
                            'cookie',
                            'url',
                            'amt',
                            'status')\
                    .withColumn('transaction_level',
                                    F.when(F.col('amt') > 300, True).otherwise(False))\
                    .withColumn('status_flag',
                                    F.when(F.col('status') == "выдана", True).otherwise(False))\
                    .withColumn('partition_date', F.lit(date).cast('string').alias('partition_date'))\
                    .withColumn('load_date', F.lit(date).cast('string').alias('load_date'))\
                    .drop('amt', 'status')
    
    datamart.write.mode("append").partitionBy('partition_date').csv('data_lake/debit_card', header=True)



start = '2024-07-01'
end = '2024-07-10'

for dt in pd.date_range(start, end):
    date = str(dt)[:10]
    print(date)
    debit_card(date)

2024-07-01
2024-07-02
2024-07-03
2024-07-04
2024-07-05
2024-07-06
2024-07-07
2024-07-08
2024-07-09
2024-07-10


In [107]:
PATH = '/home/jovyan/source'
date = '2024-07-01'


card = spark.read.csv(f"{PATH}/card.csv", header=True, sep=";")\
            .where(f''' load_date = "{date}" ''')

status_card = spark.read.csv(f"{PATH}/Status_card.csv", header=True, sep=";")\
                    .where(f''' load_date = "{date}"  ''')

transactions = spark.read.csv(f"{PATH}/transactions.csv", header=True, sep=";")\
                    .where(f''' load_date = "{date}"  ''')


In [108]:
first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx.show()

first_trx.count()

+----------------+--------------------+
|        card_num|transaction_datetime|
+----------------+--------------------+
|0119300157687351| 2024-07-02 06:32:10|
|0172264584552366| 2024-07-02 11:38:10|
|0175736131090843| 2024-07-03 01:00:10|
|0243084951118144| 2024-07-02 22:06:10|
|0261912375736877| 2024-07-02 20:35:10|
|0265281487020131| 2024-07-02 20:16:10|
|0273241962063845| 2024-07-02 21:35:10|
|0279120596277629| 2024-07-02 18:27:10|
|0284862617881895| 2024-07-02 10:19:10|
|0330327384276660| 2024-07-02 16:26:10|
|0352288633871827| 2024-07-02 02:14:10|
|0363957946193526| 2024-07-02 11:24:10|
|0385277165882644| 2024-07-02 14:36:10|
|0407156816322679| 2024-07-02 05:10:10|
|0423001802153299| 2024-07-02 18:37:10|
|0458751505613552| 2024-07-02 11:06:10|
|0458771495029206| 2024-07-02 23:31:10|
|0478546133824549| 2024-07-02 12:32:10|
|0484391462295158| 2024-07-02 07:59:10|
|0576342477104189| 2024-07-02 14:15:10|
+----------------+--------------------+
only showing top 20 rows



497

In [109]:
first_trx_info = first_trx.join(transactions, 'card_num', 'inner')

first_trx_info.where(''' card_num = "0005538167587579" ''').show()

+--------+--------------------+------+--------------------+---------+
|card_num|transaction_datetime|amount|transaction_datetime|load_date|
+--------+--------------------+------+--------------------+---------+
+--------+--------------------+------+--------------------+---------+



In [110]:
first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
          
first_trx_info.where(''' card_num = "0005538167587579" ''').show()

+--------+--------------------+------+---------+
|card_num|transaction_datetime|amount|load_date|
+--------+--------------------+------+---------+
+--------+--------------------+------+---------+



In [111]:
dt_trx = first_trx_info.select('card_num', 'amount', '*')
df_st = status_card.select('card_num', 'card_num_md5', 'status')


result_df = dt_trx\
                .join(df_st, "card_num", "inner")\
                .join(card, "card_num_md5", "right")\
                .drop('card_num_md5')


total_rows = result_df.select('card_num').count()
total_unique_rows = result_df.select('card_num').distinct().count()

print(total_rows, total_unique_rows)

1003 498


In [112]:
result_df.where(''' amount IS NULL ''').show()

+--------+------+--------------------+------+---------+------+-------------------+--------------------+----------+----------+
|card_num|amount|transaction_datetime|amount|load_date|status|      card_order_dt|                 url|    cookie| load_date|
+--------+------+--------------------+------+---------+------+-------------------+--------------------+----------+----------+
|    NULL|  NULL|                NULL|  NULL|     NULL|  NULL|2024-07-02 13:37:00|http://example.co...|v0DMPhGJqx|2024-07-02|
|    NULL|  NULL|                NULL|  NULL|     NULL|  NULL|2024-07-02 11:02:00|http://example.co...|wXDtAqpNwt|2024-07-02|
|    NULL|  NULL|                NULL|  NULL|     NULL|  NULL|2024-07-02 00:36:00|http://example.co...|7CN1gM2wzD|2024-07-02|
|    NULL|  NULL|                NULL|  NULL|     NULL|  NULL|2024-07-02 10:18:00|http://example.co...|pt0pHcuymk|2024-07-02|
|    NULL|  NULL|                NULL|  NULL|     NULL|  NULL|2024-07-02 03:20:00|http://example.co...|LpSTn3VTaS|2024

In [113]:
result_df.where(''' card_num IS NOT NULL  ''').count()

500

In [114]:
result_df.where(''' card_num = "0005538167587579"  ''').show(truncate=False)

+--------+------+--------------------+------+---------+------+-------------+---+------+---------+
|card_num|amount|transaction_datetime|amount|load_date|status|card_order_dt|url|cookie|load_date|
+--------+------+--------------------+------+---------+------+-------------+---+------+---------+
+--------+------+--------------------+------+---------+------+-------------+---+------+---------+



In [115]:
result_df.where(''' card_num = "2070700245113982"  ''').show(truncate=False)

+--------+------+--------------------+------+---------+------+-------------+---+------+---------+
|card_num|amount|transaction_datetime|amount|load_date|status|card_order_dt|url|cookie|load_date|
+--------+------+--------------------+------+---------+------+-------------+---+------+---------+
+--------+------+--------------------+------+---------+------+-------------+---+------+---------+



In [116]:
final_df = result_df\
                .where(''' card_num IS NOT NULL ''')\
                .groupBy('card_num',
                         'transaction_datetime',
                         'status',
                         'card_order_dt',
                         'url',
                         'cookie')\
                .agg(F.max('amount').alias('amt'))

final_df.show(truncate=False)

+----------------+--------------------+------+-------------------+-------------------------+----------+------+
|card_num        |transaction_datetime|status|card_order_dt      |url                      |cookie    |amt   |
+----------------+--------------------+------+-------------------+-------------------------+----------+------+
|0119300157687351|2024-07-02 06:32:10 |выдана|2024-07-02 05:56:00|http://example.com/page6 |387PbPJnn6|495.92|
|0172264584552366|2024-07-02 11:38:10 |выдана|2024-07-02 10:50:00|http://example.com/page8 |hYNBIsxI3C|127.58|
|0175736131090843|2024-07-03 01:00:10 |выдана|2024-07-02 23:35:00|http://example.com/page10|5ik2aVEYrz|230.64|
|0243084951118144|2024-07-02 22:06:10 |выдана|2024-07-02 21:06:00|http://example.com/page3 |6yojZuAfOt|124.68|
|0261912375736877|2024-07-02 20:35:10 |выдана|2024-07-02 20:10:00|http://example.com/page5 |6Cna4zRUVh|263.69|
|0265281487020131|2024-07-02 20:16:10 |выдана|2024-07-02 19:58:00|http://example.com/page6 |uwKtIsrlT9|480.28|
|

In [123]:
datamart = final_df\
                .select('card_order_dt',
                        'card_num',
                        'cookie',
                        'url',
                        'amt',
                        'status')\
                .withColumn('transaction_level',
                                F.when(F.col('amt') > 300, True).otherwise(False))\
                .withColumn('status_flag',
                                F.when(F.col('status') == "выдана", True).otherwise(False))\
                .withColumn('partition_date', F.lit(date).cast('string').alias('partition_date'))\
                .withColumn('load_date', F.lit(date).cast('string').alias('load_date'))\
                .drop('amt', 'status')

datamart.show(truncate=False)

+-------------------+----------------+----------+-------------------------+-----------------+-----------+--------------+----------+
|card_order_dt      |card_num        |cookie    |url                      |transaction_level|status_flag|partition_date|load_date |
+-------------------+----------------+----------+-------------------------+-----------------+-----------+--------------+----------+
|2024-07-02 05:56:00|0119300157687351|387PbPJnn6|http://example.com/page6 |true             |true       |2024-07-02    |2024-07-02|
|2024-07-02 10:50:00|0172264584552366|hYNBIsxI3C|http://example.com/page8 |false            |true       |2024-07-02    |2024-07-02|
|2024-07-02 23:35:00|0175736131090843|5ik2aVEYrz|http://example.com/page10|false            |true       |2024-07-02    |2024-07-02|
|2024-07-02 21:06:00|0243084951118144|6yojZuAfOt|http://example.com/page3 |false            |true       |2024-07-02    |2024-07-02|
|2024-07-02 20:10:00|0261912375736877|6Cna4zRUVh|http://example.com/page5 |f

In [118]:
datamart.write.mode("append").partitionBy('partition_date').csv('data_lake/debit_card', header=True)

In [142]:
rm -rf data_lake/debit_card/

In [124]:
datamart.printSchema()

root
 |-- card_order_dt: string (nullable = true)
 |-- card_num: string (nullable = true)
 |-- cookie: string (nullable = true)
 |-- url: string (nullable = true)
 |-- transaction_level: boolean (nullable = false)
 |-- status_flag: boolean (nullable = false)
 |-- partition_date: string (nullable = false)
 |-- load_date: string (nullable = false)



### Check

In [120]:
first_trx_info.select('card_num').groupBy('card_num')\
            .agg(F.count('*').alias("total_rows"))\
            .orderBy(F.col('total_rows').desc()).show()




first_trx = transactions.groupBy('card_num')\
                        .agg(F.min('transaction_datetime').alias('transaction_datetime'))

first_trx_info = first_trx.join(transactions, ['card_num','transaction_datetime'], 'inner')
          
first_trx_info.where(''' card_num = "2070700245113982" ''').show()



first_trx_info = first_trx.join(transactions, 'card_num', 'inner')
first_trx_info.where(''' card_num = "2070700245113982" ''').show()

+----------------+----------+
|        card_num|total_rows|
+----------------+----------+
|2070700245113982|         2|
|2262350645675539|         2|
|3103433078006244|         2|
|4348133075638360|         2|
|4544055876664150|         2|
|6242600491786411|         2|
|7787314985064344|         2|
|0005538167587579|         1|
|0007178749480464|         1|
|0071204528664757|         1|
|0089135673710847|         1|
|0092333514392220|         1|
|0117477741827069|         1|
|0123869799811751|         1|
|0131542214103394|         1|
|0140866230289371|         1|
|0153027993228699|         1|
|0162240485103303|         1|
|0162466916966431|         1|
|0192260687950694|         1|
+----------------+----------+
only showing top 20 rows

+----------------+--------------------+------+----------+
|        card_num|transaction_datetime|amount| load_date|
+----------------+--------------------+------+----------+
|2070700245113982| 2024-07-01 09:07:10| 31.11|2024-07-01|
|2070700245113982| 202